In [1]:
from langchain.document_loaders import UnstructuredPDFLoader
import PyPDF2
import fitz
import pdfrw
from pdfrw import PdfReader

file = "../../Downloads/CFA Level 1 Volume 1 (2023, CFA Institute) - libgen.li.pdf"

In [2]:
# reader = fitz.open(file)
# print(reader.load_page(0))
# print(reader.load_page(12).get_text("text"))

In [3]:
def extract_toc(file_path):
    doc = fitz.open(file_path)
    toc = doc.get_toc()
    toc_entries = []
    for level, title, page_number in toc:
        toc_entries.append({"level": level, "title": title, "page_number": page_number})
    return toc_entries


In [4]:
def remove_child_content(node):
    if not node["children"]:
        return node["content"]

    child_content = ""
    for child in node["children"]:
        child_content += remove_child_content(child)

    node["content"] = node["content"].replace(child_content, "")
    return child_content


import re

def build_nested_toc_with_content(file_path, toc):
    nested_toc = []
    stack = []

    pdf = fitz.open(file_path)

    for i, entry in enumerate(toc):
        level = entry["level"]
        start_page = entry["page_number"] - 1
        end_page = toc[i + 1]["page_number"]  - 1 if i < len(toc) - 1 else len(pdf)

        content = ""

        for page in range(start_page, end_page + 1):
            if page < len(pdf):
                content += pdf.load_page(page).get_text()

        if level == 1:
            title = f"LearningModule{i+1}"
        else:
            title = entry["title"]

        # Split content by the child's title
        if i < len(toc) - 1 and toc[i + 1]["page_number"] == entry["page_number"]:
            child_title = toc[i + 1]["title"].upper()
            child_title_regex = re.compile(r'{}'.format(child_title))
            match = child_title_regex.search(content)
            if match:
                content = content[:match.start()]

        if i < len(toc) - 1 and toc[i + 1]["page_number"] > entry["page_number"]:

            child_title = re.escape(toc[i + 1]["title"].upper())
            child_title_regex = re.compile(r'{}'.format(child_title))
            child_match = child_title_regex.search(content)

            upper_title = re.escape(title.upper())
            title_regex = re.compile(r'{}'.format(upper_title))
            match = title_regex.search(content)

            if match and child_match:
                content = content[match.start():child_match.start()]
            elif match:
                content = content[match.start():]

        # else:
        #     if i < len(toc) - 1:
        #         child_title = toc[i + 1]["title"]
        #         child_title_regex = re.compile(r'\b{}\b'.format(child_title), re.IGNORECASE)
        #         match = child_title_regex.search(content)
        #         if match:
        #             content = content[match.start():]

        item = {
            "title": title,
            "page_number": entry["page_number"],
            "content": content,
            "children": []
        }

        while len(stack) >= level:
            stack.pop()

        if not stack:
            nested_toc.append(item)
        else:
            parent = stack[-1]
            parent["children"].append(item)

        stack.append(item)

    pdf.close()

    # Remove content of child nodes from parent nodes
    for item in nested_toc:
        remove_child_content(item)

    return nested_toc



toc = extract_toc(file)
sections = build_nested_toc_with_content(file, toc)

In [10]:
print(sections[1]["children"][0]['children'][8]['content'])

Present Value of a Series of Equal and Unequal Cash Flows
21
PRESENT VALUE OF A SERIES OF EQUAL AND 
UNEQUAL CASH FLOWS
calculate and interpret the future value (FV) and present value (PV) 
of a single sum of money, an ordinary annuity, an annuity due, a 
perpetuity (PV only), and a series of unequal cash flows
demonstrate the use of a time line in modeling and solving time 
value of money problems
Many applications in investment management involve assets that offer a series of 
cash flows over time. The cash flows may be highly uneven, relatively even, or equal. 
They may occur over relatively short periods of time, longer periods of time, or even 
stretch on indefinitely. In this section, we discuss how to find the present value of a 
series of cash flows.
The Present Value of a Series of Equal Cash Flows
We begin with an ordinary annuity. Recall that an ordinary annuity has equal annuity 
payments, with the first payment starting one period into the future. In total, the 
annuity ma

[{'level': 1, 'title': 'How to Use the CFA Program Curriculum', 'page_number': 9}, {'level': 2, 'title': 'Errata', 'page_number': 9}, {'level': 3, 'title': 'Designing Your Personal Study Program', 'page_number': 9}, {'level': 3, 'title': 'CFA Institute Learning Ecosystem (LES)', 'page_number': 10}, {'level': 3, 'title': 'Feedback', 'page_number': 10}, {'level': 1, 'title': 'Quantitative Methods', 'page_number': 11}, {'level': 2, 'title': 'Learning Module 1\tThe Time Value of Money', 'page_number': 13}, {'level': 3, 'title': 'Introduction', 'page_number': 13}, {'level': 3, 'title': 'Interest Rates', 'page_number': 14}, {'level': 3, 'title': 'Future Value of a Single Cash Flow', 'page_number': 16}, {'level': 3, 'title': 'Non-Annual Compounding (Future Value)', 'page_number': 20}, {'level': 3, 'title': 'Continuous Compounding', 'page_number': 22}, {'level': 4, 'title': 'Stated and Effective Rates', 'page_number': 24}, {'level': 3, 'title': 'A Series of Cash Flows', 'page_number': 25}, {'l

In [130]:
sections[1]

{'title': 'LearningModule6',
 'page_number': 11,
 'content': 'Quantitative Methods\n© CFA Institute. For candidate use only. Not for distribution.\n© CFA Institute. For candidate use only. Not for distribution.\nThe Time Value of Money\nby Richard A. DeFusco, PhD, CFA, Dennis W. McLeavey, DBA, CFA, Jerald \nE. Pinto, PhD, CFA, and David E. Runkle, PhD, CFA.\nRichard A. DeFusco, PhD, CFA, is at the University of Nebraska-Lincoln (USA). Dennis W. \nMcLeavey, DBA, CFA, is at the University of Rhode Island (USA). Jerald E. Pinto, PhD, \nCFA, is at CFA Institute (USA). David E. Runkle, PhD, CFA, is at Jacobs Levy Equity \nManagement (USA).\nLEARNING OUTCOME\nMastery\nThe candidate should be able to:\ninterpret interest rates as required rates of return, discount rates, or \nopportunity costs\nexplain an interest rate as the sum of a real risk-free rate and \npremiums that compensate investors for bearing distinct types of \nrisk\ncalculate and interpret the future value (FV) and present val

In [23]:
def remove_content_until_learning_outcome(content):
    match = re.search(r'LEARNING OUTCOME', content)
    if match:
        content = content[match.start():]

    match = re.search(r'(L\sE\sA\sR\sN\sI\sN\sG\sM\sO\sD\sU\sL\sE)', content)
    if match:
        content = content[:match.end() + 1]

    return content

In [24]:
def test_remove_content_until_learning_outcome():
    input_text = """This text should be removed. LEARNING OUTCOME Mastery The candidate should be able to: interpret interest rates as required rates of return, Therefore, a smaller amount 1 L E A R N I N G M O D U L E blah blah blah blah this text should be removed"""

    expected_output = """LEARNING OUTCOME Mastery The candidate should be able to: interpret interest rates as required rates of return, Therefore, a smaller amount 1 L E A R N I N G M O D U L E"""

    output = remove_content_until_learning_outcome(input_text)
    assert output == expected_output, f"Expected: {expected_output}, but got: {output}"

test_remove_content_until_learning_outcome()


AssertionError: Expected: LEARNING OUTCOME Mastery The candidate should be able to: interpret interest rates as required rates of return, Therefore, a smaller amount 1 L E A R N I N G M O D U L E, but got: LEARNING OUTCOME Mastery The candidate should be able to: interpret interest rates as required rates of return, Therefore, a smaller amount 1 L E A R N I N G M O D U L E 

In [17]:
test_remove_content_until_learning_outcome()

In [1]:
import sympy

In [146]:

formula = "FVN = PV(1 + r)N"
lhs, rhs = formula.split("=")
rhs = rhs.replace(" ", "").replace("(", "*(")  # Add a multiplication symbol before the parentheses
expr = sympy.sympify(f"{lhs.strip()} - ({rhs.strip()})")

In [ ]:
formula = "FVN = PV(1 + r)N"
lhs, rhs = formula.split("=")
rhs = rhs.replace(" ", "").replace("(", "*(")  # Add a multiplication symbol before the parentheses
expr = sympy.sympify(f"{lhs.strip()} - ({rhs.strip()})")

# Solve the equation for FVN
FVN = sympy.Symbol('FVN')
solved_expr = sympy.solve(expr, FVN)[0]

# Convert the expression to LaTeX
latex_formula = sympy.latex(solved_expr)